In [1]:
%matplotlib inline
from IPython.display import Image

# 強化學習Cartpole (cntk)

相較於之前學習過的機器視覺與自然語言，強化學習是另一種不同的人工智能方向。機器視覺與語言注重的是如何透過神經網路結構來實現人類視覺以及語言理解的內部機制。但是強化學習卻不關心這些，強化學習的本質是一種行為控制的科學，它關心的是如何能夠控制智能體的行為模式，但卻不需要關注於智能體實現這個行為模式的內部結構。舉例來說，傳統的技術如果要讓智能體懂得「跑」這件事，他需要對於人類的運動機制、肌肉控制等過程需要有巨細靡遺理解，但是也正是這些阻礙地讓機器跑起來的可能性，因為這裡涉及到太多的內部機制。而強化學習的目標則是透過外部的獎賞與懲罰，透過智能體與環境間的交互，來誘導智能體自然產生「跑」這個行為。


![md_images](../Images/rl.jpg)

## CartPole: 數據與環境

在本次實作中，我們將會來介紹強化學習的經典案例Cartpole。我們將使用OpenAI's [gym](https://github.com/openai/gym)模擬器中的[CartPole](https://gym.openai.com/envs/CartPole-v0)環境來做為我們智能體所存在以及交互的世界，它會不斷的反饋給我們木棒與台車的狀態。

環境給予我們的狀態(state)是一個長度為4的向量 $(x, \dot{x}, \theta, \dot{\theta})$, 分別表示 *台車位置*, *台車速度*, *木棒對垂直交角*, *木棒角速度(落下速度)*,

### CartPole: 行動策略
至於台車為了不讓棒子掉落，必須採取對應行動，它能夠執行的行動只有兩種: `向左` 或 `向右`

### CartPole: 環境交互
當台車執行了某個行動後，它可從環境中獲得
  * 每多存活一刻就會獲得獎賞 +1 
  * 新狀態 $(x', \dot{x}', \theta', \dot{\theta}')$

### CartPole: 失敗的定義
這也正是「活得越久，領得越多」，那麼機器的目標很明確，就是希望採取正確的行動，來讓木棒可以撐久一點不要掉下去。那至於甚麼樣的情況會導致木棒掉落呢？如果觸發以下條件，則episode終止
 * 木棒距垂直角度大於15度
 * 臺車移動距離中心超過2.4個單位

### CartPole: 成功的定義
我們對於台車的任務給予一個任務完成認定條件如下：
 * 智能體在過去50個episodes期間累積獎賞值200以上

### CartPole: 優化的目標
如果用RL的術語來說，整個模型的目標就是要找尋 _行動策略(向左向右)_ $a$, 藉由與環境的互動(讓木棒保持平衡)好讓 _獎賞_ $r$ 最大化。於是給定一系列的實驗 $$s \xrightarrow{a} r, s'$$ 讓後讓智能體學會在給定的狀態下 $s$ 找出最佳的行動策略 $a$ 以將跨時間累積獎賞最大化 $r$ :

$$
Q(s,a) = r_0 + \gamma r_1 + \gamma^2 r_2 + \ldots = r_0 + \gamma \max_a Q^*(s',a)
$$

此處的 $\gamma \in [0,1)$ 是波爾曼方程式 [*Bellmann*-equation] 用來控制評估未來獎賞的折價因子，我們也稱之為(https://en.wikipedia.org/wiki/Bellman_equation).

在接下來的範例中，我們將示範如何針對狀態空間建模，以及如何根據收到的獎賞，來轉化為取得未來最大獎賞的行動。

![md_images](../Images/polecart.gif)

在此我們將採用兩種常見的技術:

**Deep Q-Networks (DQN)**: DQNs在2015年只透過遊戲畫面像素數據就能訓練智能體玩Atari電動遊戲而聲名大噪。我們訓練神經網路學習 $Q(s,a)$ 值 (也就是 $Q$-Network )，你可以把Q函數想像為一個可以根據目前狀態去查詢各個行動策略對應的可能獎賞值(目前獎賞與預估未來獎賞)的對應表，因為這個表可能很複雜或是根本不存在，所以我們需要用一個網路去近似它。根據這 $Q$ 函數值，我們交會逐一評估所有可能的策略，選擇獎賞最高的為最佳策略。(我常開玩笑說DQN是最視錢如命的模型，它的輸入是視覺，輸出是怎麼做會最賺錢)，它的輸出既然是獎賞的期望值，所以它的本質是一種迴歸。

**Policy gradient (策略梯度)**: 這個方法是在神經網路中直接估計策略(行動組合)的機率分布，通過機率選擇動作的子集來最大化獎勵，所以你可以把它視為是一種分類模型，輸出的是各個行動的分布機率。 

請注意，由於這個實作的目的是為了要理解RL的概念，因此我們網路部分都使用簡單的淺層網路，當然這個部分是可以日後再擴充與使用結構更複雜的網路。

In [2]:
from __future__ import print_function
from __future__ import division
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pandas as pd
import seaborn as sns
import numpy
import math 
import os 
import random
import cntk as C
from cntk.device import try_set_default_device, cpu, gpu

style.use('ggplot')
%matplotlib inline


# 是否使用GPU
is_gpu = True

if is_gpu:
    try_set_default_device(gpu(0))
else:
    try_set_default_device(cpu())

執行實作前需要安裝 OpenAI gym包

In [3]:
try:
    import gym
except:
    !pip install gym
    import gym

## 強化學習方法1: DQN

DQN的主要核心在於收集歷史案例的四元組$(s,a,r,s')$ ，這四個分別是之前的狀態，執行的行動、得到的獎賞以及執行後的新狀態，透過這四元祖，我們可以利用深度學習網路去近似價值函數$Q(s,a)$，這個價值函數$Q(s,a)$的實際值應該要接近 $r+\gamma \max_{a'}Q(s',a')$, 其中 $\gamma$ 是未來獎勵的折扣因子，值介於0和1之間。這種先收集樣本，然後在採樣後建模訓練的手法稱之為「回放(replay)」，可以有效地換解如果我們按照數據的時序訓練時，容易偏重當下的案例，反而造成樣本的不均衡。


### Model: DQN

$$
l_1 = relu( x W_1 + b_1) \\
Q(s,a) = l_1 W_2 + b_2 \\
$$

來源參考自Keras版本實現, https://github.com/jaara/AI-blog/blob/master/CartPole-basic.py, 作者是Jaromír Janisch 首見於他的 [AI blog](https://jaromiru.com/2016/09/27/lets-make-a-dqn-theory/)。

首先利用gym建置CartPole-v0模擬器環境，狀態空間長度為4，行動策略長度為2

STATE_COUNT = 4 (對應至 $(x, \dot{x}, \theta, \dot{\theta})$),
ACTION_COUNT = 2 (對應至`向左` or `向右`)

In [4]:
env = gym.make('CartPole-v0')

STATE_COUNT  = env.observation_space.shape[0]
ACTION_COUNT = env.action_space.n

STATE_COUNT, ACTION_COUNT

(4, 2)

在此，我們使用簡單的兩層全連接層網路來做示範，用來逼近價值函數$Q(s,a)$。如果你覺得兩層太簡單了，你也可以自行設計或者是置換為更複雜的網路結構。

![md_images](../Images/dqn_network.jpg)

In [5]:
# 目標獎賞
REWARD_TARGET =  200
# 平均episodes數
BATCH_SIZE_BASELINE = 50



#大腦類別
class Brain:
    def __init__(self):
        self.params = {}
        self.model, self.trainer, self.loss = self._create()
    #大腦中主要是透過兩層全連階層來近似Q函數
    def _create(self):
        observation = C.sequence.input_variable(STATE_COUNT, np.float32, name="s")  #狀態
        q_target = C.sequence.input_variable(ACTION_COUNT, np.float32, name="q") #行動價值
        
        #使用兩層全連階層建構DQN
        l1 = C.layers.Dense(24, activation=C.relu)
        l2 = C.layers.Dense(64, activation=C.relu)
        l3 = C.layers.Dense(ACTION_COUNT)
        unbound_model = C.layers.Sequential([l1,l2,l3])
        model = unbound_model(observation)

        # loss='mse'
        #模型預測的各個行動的r值應該要接近真實
        loss = C.reduce_mean(C.square(model - q_target), axis=0)
       
        # 最佳化
        lr = 0.001
        lr_schedule = C.learning_parameter_schedule(lr)
        learner = C.sgd(model.parameters, lr_schedule, gradient_clipping_threshold_per_sample=10)
        trainer = C.Trainer(model, (loss, loss), learner)
        return model, trainer, loss

    def train(self, x, y, epoch=1, verbose=0):
        arguments = dict(zip(self.loss.arguments, [x,y]))
        updated, results =self.trainer.train_minibatch(arguments, outputs=[self.loss.output])

    def predict(self, s):
        return self.model.eval([s])

 `記憶(Memory)` 類別是專門用來儲存歷史的四元組紀錄。(之前的狀態，執行的行動、得到的獎賞以及執行後的新狀態)

In [6]:
class Memory:   #儲存 ( s, a, r, s_ )
    samples = []

    def __init__(self, capacity):
        self.capacity = capacity

    def add(self, sample):
        self.samples.append(sample)

        if len(self.samples) > self.capacity:
            self.samples.pop(0)
    
    #從記憶中隨機取最近n筆
    def sample(self, n):
        n = min(n, len(self.samples))
        return random.sample(self.samples, n)

`智能體` 使用 `Brain` 以及 `Memory` 來重放(replay)過去的行動以訓練出能讓獎賞最大化的行動集合。

In [7]:
MEMORY_CAPACITY = 100000
BATCH_SIZE = 64

GAMMA =  0.95 # 折價因子

MAX_EPSILON = 1  #初期還沒有案例可以供建模，因此100%根據隨機案例
MIN_EPSILON = 0.01 # 即使模型準確率越來越高，還是必須保留部分比例基於隨機案例
LAMBDA = 0.0001    # 衰減速度

#智能體類別
class Agent:
    steps = 0
    epsilon = MAX_EPSILON

    def __init__(self):
        #裡面主要有大腦以及記憶
        #大腦裡具有輸入為狀態，輸出為各個行動的獎賞數值的神經網路
        #記憶負責儲存四元組以及抽取樣本
        self.brain = Brain()
        self.memory = Memory(MEMORY_CAPACITY)

    def act(self, s):
        if random.random() < self.epsilon:
            #部分基於隨機行動
            return random.randint(0, ACTION_COUNT-1)
        else:
            #部分基於模型預測(這部分比率會逐漸增高)
            return numpy.argmax(self.brain.predict(s))

    def observe(self, sample):  # in (s, a, r, s_) format
        #當觀察到新案例先把他加入記憶中
        self.memory.add(sample)
        self.steps += 1
        # 一開始需要更多的探索，所以動作偏隨機，慢慢的我們開始能夠掌握訣竅，因此減少隨機。
        self.epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * math.exp(-LAMBDA * self.steps)

    #回放
    def replay(self):
        #取最近n筆作為minibatch
        batch = self.memory.sample(BATCH_SIZE)
        batchLen = len(batch)
        no_state = numpy.zeros(STATE_COUNT)

        #從四元組中抽取「初始狀態」
        states = numpy.array([ o[0] for o in batch ], dtype=np.float32)
        #從四元組中抽取「行動後新狀態」
        states_ = np.array([(no_state if o[3] is None else o[3]) for o in batch ], dtype=np.float32)

        #根據目前狀態預測的目前獎賞
        p = agent.brain.predict(states)
        #根據行動後新狀態預測的未來獎賞
        p_ = agent.brain.predict(states_)

        x = np.zeros((batchLen, STATE_COUNT)).astype(np.float32)
        y = np.zeros((batchLen, ACTION_COUNT)).astype(np.float32)

        #組出批次的x,y
        for i in range(batchLen):
            s, a, r, s_ = batch[i]

            t = p[0][i]
            
            if s_ is None:
                t[a] = r
            else:
                t[a] = r + GAMMA * np.amax(p_[0][i])

            x[i] = s
            y[i] = t

        self.brain.train(x, y)

接下來就可以開始訓練智能體的 **DQN**。請注意，若要達到 50 batches平均獎賞值達200以上，需要耗時約10分鐘。

In [ ]:
TOTAL_EPISODES = 3000

#定義執行智能體函數
def run(agent):
    #將環境重置
    s = env.reset()
    #獎賞歸零
    R = 0

    while True:
        #env.render()指令會幫我們渲染出目前台車狀態圖形
        env.render()
        #agent.act(s.astype(np.float32))表示是根據目前狀態產生對應行動
        #在這邊act函數初期是基於隨機行動，後面隨機行動站筆會逐漸下降
        a = agent.act(s.astype(np.float32))
        #根據此行動，產出對應的獎賞以及新的狀態
        s_, r, done, info = env.step(a)

        if done: # 若最後是終止狀態(倒下)，則新狀態為None
            s_ = None
        
        #構成四元組儲存在記憶中
        agent.observe((s, a, r, s_))
        #根據隨機抽樣進行回放訓練
        agent.replay()
        
        #將狀態更新至新狀態
        s = s_
        #將獎賞累加
        R += r

        # 若最後是終止狀態(倒下)，則獎賞不再更新
        if done:
            return R

以下為訓練語法

In [ ]:
agent = Agent()#宣告智能體

episode_number = 0
reward_sum = 0
while episode_number < TOTAL_EPISODES:
    #執行智能體一輪一直到倒下為止，記錄這一輪的累計獎賞
    reward_sum += run(agent)
    episode_number += 1
    if episode_number % BATCH_SIZE_BASELINE == 0:
        print('Episode: {0}, episode的平均獎賞為{1} '.format(episode_number,reward_sum / BATCH_SIZE_BASELINE))
        reward_sum = 0
        agent.brain.model.save('Models/dqn_cntk.model')


Episode: 50, episode的平均獎賞為21.24 
Episode: 100, episode的平均獎賞為22.06 
Episode: 150, episode的平均獎賞為20.98 
Episode: 200, episode的平均獎賞為40.78 
Episode: 250, episode的平均獎賞為66.06 
Episode: 300, episode的平均獎賞為149.68 
Episode: 350, episode的平均獎賞為200.0 


### 執行DQN模型

前面將模型訓練好之後我們就可以開始來測試訓練好的DQN模型。

In [ ]:
env = gym.make('CartPole-v0')

num_episodes = 10  #欲執行的批次數

#換原訓練好的模型
root = C.load_model('Models/dqn_cntk.model')

for i_episode in range(num_episodes):
    observation = env.reset()  #每次episode開始歸零重置
    done = False
    n=0
    tot_reward=0
    while not done: 
        env.render()
        #只根據模型預測結果進行act決定
        action = np.argmax(root.eval([observation.astype(np.float32)]))
        observation, reward, done, info  = env.step(action)  
        #累加獎賞
        tot_reward+=reward
        n+=1
    print('Episode {0} 累積獎賞:{0}'.format(i_episode,tot_reward))